In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
from NeuralNetwork.torch_cnn import MEL_CNN, CHR_CNN

In [3]:
# 图像预处理
mel_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

# 加载数据集
train_dataset = datasets.ImageFolder(root="./dataset/mel_format", transform=mel_transform)
test_dataset = datasets.ImageFolder(root="./dataset/mel_test", transform=mel_transform)

train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=100, shuffle=False)

In [4]:
# 初始化模型
device = torch.device("cuda")
model = MEL_CNN().to(device)

In [6]:
# 损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# 训练模型
weight = 5
num_epochs = 15
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f'Epoch {epoch+1}, Loss: {total_loss/len(train_loader)}')

    # 在测试集上评估模型
    if epoch % weight == 4:
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        print(f'Validation Accuracy: {100 * correct / total}%')

# 保存模型
torch.save(model.state_dict(), './model/torch_mel_model.pth')

Epoch 1, Loss: 2.170912575721741
Epoch 2, Loss: 1.999832694530487
Epoch 3, Loss: 1.8906725883483886
Epoch 4, Loss: 1.8237228441238402
Epoch 5, Loss: 1.7739182615280151
Validation Accuracy: 39.23923923923924%
Epoch 6, Loss: 1.7332324171066285
Epoch 7, Loss: 1.7026690912246705
Epoch 8, Loss: 1.6733091807365417
Epoch 9, Loss: 1.651175284385681
Epoch 10, Loss: 1.6078662538528443
Validation Accuracy: 76.87687687687688%
Epoch 11, Loss: 1.562927677631378
Epoch 12, Loss: 1.5346216320991517
Epoch 13, Loss: 1.5208972787857056
Epoch 14, Loss: 1.5100887894630433
Epoch 15, Loss: 1.499456331729889
Validation Accuracy: 72.17217217217217%


In [5]:
# 图像预处理
chr_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((12, 1293)),
    transforms.ToTensor(),
])

# 加载数据集
train_dataset = datasets.ImageFolder(root="./dataset/chr_format", transform=chr_transform)
train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True)

chr_model = CHR_CNN().to(device)
optimizer = optim.Adam(chr_model.parameters(), lr=0.0001)

In [23]:
# 训练模型
num_epochs = 20
for epoch in range(num_epochs):
    chr_model.train()
    total_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = chr_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f'Epoch {epoch+1}, Loss: {total_loss/len(train_loader)}')

# 保存模型
torch.save(chr_model.state_dict(), './model/torch_chr_model.pth')

Epoch 1, Loss: 1.517870795726776
Epoch 2, Loss: 1.5118595361709595
Epoch 3, Loss: 1.506114661693573
Epoch 4, Loss: 1.5025652647018433
Epoch 5, Loss: 1.4975980997085572


In [6]:
model.load_state_dict(torch.load("./model/torch_mel_model.pth"))
chr_model.load_state_dict(torch.load("./model/torch_chr_model.pth"))

<All keys matched successfully>

In [13]:
import numpy as np
import librosa
from PIL import Image
from random import randint

y, sr = librosa.load("./dataset/test_long/DSM-V.mp3")
# y, sr = librosa.load("./dataset/audio_format/classical/classical.00001.wav")

spectrogram = librosa.feature.melspectrogram(y=y, sr=sr)

mel_pic = librosa.power_to_db(spectrogram, ref=np.max)

harmonic = librosa.effects.harmonic(y)  
harmonic_features = librosa.feature.chroma_cqt(y=harmonic, sr=sr)  

# mfccs = librosa.feature.mfcc(y=y, sr=sr)

duration = len(y) / sr

mel_predictions = []
times = round(duration / 5)
for i in range(times):
    start_pos = randint((mel_pic.shape[1] - 128) // times * i, (mel_pic.shape[1] - 128) // times * (i + 1))
    img = mel_transform(Image.fromarray(mel_pic[:, start_pos:start_pos + 128])).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs: torch.Tensor = model(img)
        prediction = outputs.data.cpu().numpy()
        mel_predictions.append(prediction)

mel_predictions = np.mean(mel_predictions, axis=0)
mel_predictions[mel_predictions < 0] = 0
print(mel_predictions)
print(["pop", "classical", "pop", "pop", "pop", "jazz", "rock", "pop", "pop", "rock"][np.argmax(mel_predictions, axis=1)[0]])


chr_predictions = []
for i in range(0, round(duration), 30):
    if harmonic_features.shape[1] <= (i + 1) * 1293: 
        break
    img = chr_transform(Image.fromarray(harmonic_features[:, i * 1293:(i + 1) * 1293])).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs: torch.Tensor = chr_model(img)
        prediction = outputs.data.cpu().numpy()
        chr_predictions.append(prediction)

chr_predictions = np.mean(chr_predictions, axis=0)
chr_predictions[chr_predictions < 0] = 0
print(chr_predictions)
print(["pop", "classical", "pop", "pop", "pop", "jazz", "rock", "pop", "pop", "rock"][np.argmax(chr_predictions, axis=1)[0]])


class_index = np.argmax((chr_predictions * 2 + mel_predictions) / 3, axis=1)[0]
print(["pop", "classical", "pop", "pop", "pop", "jazz", "rock", "pop", "pop", "rock"][class_index])

[[0.01262312 0.05022861 0.00367169 0.01370685 0.05700575 0.02329349
  0.00464177 0.00993608 0.8157727  0.00911982]]
8
pop
[[0.1877748  0.03029667 0.04228276 0.18215984 0.01210932 0.19232441
  0.04015813 0.08269724 0.11689904 0.11329769]]
5
jazz
pop
